# Living Earth datasets

Example load and display code for relevant global datasets.

- [Area of interest](#Area-of-interest)
- [CCI Biomass](#CCI-Biomass)

In [ ]:
import sys
from pathlib import Path
import geopandas as gpd
# from s3path import S3Path
# import dask_geopandas as dask_gpd

# Matlplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# Holoview / Hvplot - works best for geographic projection data. If using a UTM projection, consider using matplotlib
# import hvplot.xarray
# from holoviews import opts as hvopts

# Geospatial
import rasterio
from rasterio.plot import show

import odc.geo.xr
import datacube
from datacube.utils.aws import configure_s3_access

In [ ]:
# EASI tools

repo = Path.home() / 'easi-notebooks'
sys.path.append(str(repo))
from easi_tools import unset_cachingproxy, initialize_dask

cluster, client = initialize_dask(workers=4)
display(client)

In [ ]:
dc = datacube.Datacube()

# Access AWS "requester-pays" buckets
# This is necessary for reading data from most third-party AWS S3 buckets such as for Landsat and Sentinel-2
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

## Area of interest

In [ ]:
# Target CRS and resolution
# Two options:
# 1. Load an xarray dataset into the target crs/resolution, then use its geobox as the target for future loads
# 2. Define a target geobox with a affine or geometry methods
# Using the same geobox for future dataset loads helps ensure the xarray grids are directly comparable

# Defaults - None will load with the product defaults
latitude, longitude = None, None
target_crs, target_res, gbox = None, None, None 

# PNG
latitude = (-7.88, -7.19)
longitude = (144.07, 145.09)
target_crs = "EPSG:32755"   # Target CRS - https://epsg.io/?q=Papua%20New%20Guinea%20kind%3APROJCRS
target_res = odc.geo.resxy_(30, -30)

# Bangladesh
# latitude = (22, 22.5)
# longitude = (88.5, 89.0)

In [ ]:
# Example bounding box
# poly = odc.geo.geom.box(longitude[0], latitude[0], longitude[1], latitude[1], 'epsg:4326')

# Example geopolygon
# poly = odc.geo.geom.polygon(...)

# gbox = odc.geo.geobox.GeoBox.from_geopolygon(poly, resolution = 0.0001)

# Display your efforts
# display(gbox)
# gbox.explore()

In [ ]:
# Determine target dimension names

dims = (None, None)
if gbox is not None:
    dims = gbox.dims
elif target_crs is not None:
    dims = odc.geo.CRS(target_crs).dimensions
# dims

In [ ]:
# Test that we can load a file, no dask, with or without the caching-proxy

# Set a dev caching proxy address
# import os
# env = os.environ
# env['GDAL_HTTP_PROXY'] = '10.0.48.14'

# target = 's3://easi-asia-dc-data/products-index/cci_biomass_annual_v51/N00E140/N00E140_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv5.0/N00E140_ESACCI-BIOMASS-L4-AGB-MERGED-100m-2010-fv5.0.tif'

# xx = rasterio.open(target, chunks=True)
# show(xx)

# with unset_cachingproxy():
#     xx = rasterio.open(target, chunks=True)
#     show(xx)

## CCI Biomass

In [ ]:
product = 'cci_biomass_annual_v51'

# Optional - print product/measurement details
display(dc.list_products().loc[product])
display(dc.list_measurements().loc[product])

xx = dc.load(
    product = product,
    time = ('2010', None),  # all data from 2010 onward
    like = gbox,
    latitude = latitude,
    longitude = longitude,
    output_crs = target_crs,
    resolution = target_res,
    dask_chunks = {dims[0]:2048, dims[1]:2048},   # Change chunk size depending on data request size. Target: size per chunk =~ 10s MB, Number of chunks <~ 100,000s
)
xx

In [ ]:
# Select combinations of variables and color maps

xx.agb.plot(
    y=dims[0], x=dims[1],
    col="time", col_wrap=4,
    cmap='viridis',
    robust=True,
)